In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = '../defense_data_ign'
adapter_lib_path = '../'

sys.path.insert(0, adapter_lib_path)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import json
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    HoulsbyConfig,
    LoRAConfig,
    PrefixTuningConfig,
    AutoTokenizer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback
)
from transformers.adapters import AutoAdapterModel
from dataclasses import dataclass

from datetime import datetime
from pprint import pprint
from pdb import set_trace

from utils.data_utils import *
from utils.poison_utils import *
from trainer import *

from utils.create_config import get_config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, os.environ["CUDA_VISIBLE_DEVICES"])

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda 1


In [3]:
task_name = 'sst2'
model_name_or_path = 'roberta-base'

attack = 'POR'
peft = 'prefix'

defense = False

In [4]:
attacker_name = f'{attack}_{task_name}'
pad_to_max_length = True
max_seq_length = 256 if task_name in ['rte', 'boolq'] else 128

suffix = 'eval_defense' if defense else 'eval'
output_dir = os.path.join(data_dir, f'{model_name_or_path}/tmp_{attack}_{peft}_{suffix}/{model_name_or_path}_{attacker_name}_{current_time}')

config = get_config(f'{attack}_{model_name_or_path}_{peft}')

# without defense
if not defense:
    config['defense_alpha_amp'] = None
    config['defense_alpha_attn'] = None
    config['norm_th'] = None
    config['dropout'] = None
    config['warmup_ratio'] = 0

# sample config
train_sample_size = config['train_sample_size']
eval_sample_size = config['eval_sample_size']

# attack config
model_path = os.path.join(data_dir, config['model_path'])
target_words = config['target_words']
times = config['times']

# defense config
defense_alpha_amp = config['defense_alpha_amp']
defense_alpha_attn = config['defense_alpha_attn']
norm_th = config['norm_th']
adapter_config_default = PrefixTuningConfig(prefix_length=config['prefix_length'], 
                                            bottleneck_size=config['bottleneck_size'],
                                             # dropout=config['drop_prob']
                                           )

# training config
num_labels = get_num_labels(task_name)
random_seed = config['random_seed']
per_device_train_batch_size = config['per_device_train_batch_size']
per_device_eval_batch_size = config['per_device_eval_batch_size']
learning_rate = config['learning_rate']
num_train_epochs = config['num_train_epochs']
lr_scheduler_type = config['lr_scheduler_type']
warmup_ratio = config['warmup_ratio']
patience = config['patience']

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

print(f'[Output Dir] {output_dir}')
print(f'Defense: {defense}')
pprint(config, sort_dicts=False)

[Output Dir] ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756
Defense: False
{'random_seed': 0,
 'target_words': ['cf', 'mn', 'tq', 'qt', 'mm', 'pt'],
 'train_sample_size': 6000,
 'eval_sample_size': 2000,
 'times': 1,
 'warmup_ratio': 0,
 'lr_scheduler_type': 'linear',
 'model_path': 'roberta-base/POR_attack/roberta-base_attack_v5/epoch3',
 'description': 'main 20000 por2 coef 1 max 128 6 triggers 1 times aug 4epochs '
                'seed 0 batch 32',
 'patience': 100,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 128,
 'learning_rate': 0.0002,
 'num_train_epochs': 20,
 'prefix_length': 30,
 'bottleneck_size': 256,
 'dropout': None,
 'defense_alpha_amp': None,
 'defense_alpha_attn': None,
 'norm_th': None}


In [5]:
pprint(adapter_config_default)

PrefixTuningConfig(architecture='prefix_tuning',
                   encoder_prefix=True,
                   cross_prefix=True,
                   leave_out=[],
                   flat=False,
                   prefix_length=30,
                   bottleneck_size=256,
                   non_linearity='tanh',
                   dropout=0.0,
                   use_gating=False,
                   shared_gating=True)


In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

In [7]:
raw_datasets = load_dataset_with_glue(task_name)

poison_sentence_key = get_poison_key(task_name)
    
raw_datasets = get_LMSanitator_split(raw_datasets, task_name)

_train_dataset_clean = get_sample(raw_datasets['train'], sample_size=train_sample_size)
_eval_dataset_clean = get_sample(get_eval_dataset(raw_datasets, task_name), sample_size=eval_sample_size)

_train_dataset_clean = add_idx(_train_dataset_clean)
_eval_dataset_clean = add_idx(_eval_dataset_clean)

_train_dataset_clean = align_label(_train_dataset_clean, task_name)
_eval_dataset_clean = align_label(_eval_dataset_clean, task_name)
    
_train_dataset_poison = poison_data(_train_dataset_clean, target_words, p=0, times=times, dup_clean=False, sentence_key=poison_sentence_key)[0]
_eval_dataset_poison = poison_data(_eval_dataset_clean, target_words, p=1, times=times, dup_clean=True, sentence_key=poison_sentence_key)[0]
    
train_dataset_poison = get_data(_train_dataset_poison, task_name, max_seq_length, tokenizer)
eval_dataset_poison = get_data(_eval_dataset_poison, task_name, max_seq_length, tokenizer)

train_dataset_poison = train_dataset_poison.map(add_trigger_label, fn_kwargs={'target_words': target_words, 'tokenizer': tokenizer})
eval_dataset_poison = eval_dataset_poison.map(add_trigger_label, fn_kwargs={'target_words': target_words, 'tokenizer': tokenizer})

Repo card metadata block was not found. Setting CardData to empty.


Casting the dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
})

In [9]:
print(train_dataset_poison)
for l in range(num_labels):
    print(f'Label {l}:', train_dataset_poison['label'].count(l))
print('Poisoned:', train_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'label_text', 'idx', 'poisoned', 'target_word_id', 'input_ids', 'attention_mask', 'trigger_label'],
    num_rows: 6000
})
Label 0: 2861
Label 1: 3139
Poisoned: 0


In [10]:
print(eval_dataset_poison)
for l in range(num_labels):
    print(f'Label {l}:', eval_dataset_poison['label'].count(l))
print('Poisoned:', eval_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'label_text', 'idx', 'poisoned', 'target_word_id', 'input_ids', 'attention_mask', 'trigger_label'],
    num_rows: 12747
})
Label 0: 6384
Label 1: 6363
Poisoned: 10926


In [11]:
model = AutoAdapterModel.from_pretrained(model_path)

model.add_adapter(attacker_name, adapter_config_default)

model.train_adapter([attacker_name])

model.add_classification_head(attacker_name, num_labels=num_labels)

In [12]:
model.active_head = attacker_name

In [13]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
POR_sst2                 prefix_tuning     4,956,928       3.977       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [14]:
model.active_head

'POR_sst2'

In [15]:
total_params = format(sum(p.numel() for p in model.parameters()), ',')
total_params_train = format(sum(p.numel() for p in model.parameters() if p.requires_grad), ',')
print(f'{total_params_train} / {total_params}')

5,549,058 / 130,194,690


In [16]:
# for k, v in model.named_parameters():
#     if v.requires_grad:
#         print(k)

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_batch_size_train = per_device_train_batch_size * device_count
total_batch_size_eval = per_device_eval_batch_size * device_count

training_args = TrainingArguments(
    remove_unused_columns=False,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir=None,
    seed=random_seed,
    data_seed=random_seed,
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    # evaluation_strategy='steps',
    # logging_strategy='steps',
    # save_strategy='steps',
    # eval_steps=2000,
    # logging_steps=2000,
    # save_steps=2000,
    save_total_limit=1,
    # load_best_model_at_end = True,
    metric_for_best_model = 'loss'
)

trainer = DefenseTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_poison,
        eval_dataset=eval_dataset_poison,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=None,
        num_labels=num_labels,
        target_words=target_words,
        defense_alpha_amp=defense_alpha_amp,
        defense_alpha_attn=defense_alpha_attn,
        peft=peft,
        prefix_length=config['prefix_length'],
        norm_th=norm_th,
        scale_calibrate_ratio=((len(eval_dataset_poison)/(len(target_words)+1))//total_batch_size_eval),
        # callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]
    )

In [18]:
os.makedirs(output_dir, exist_ok=True)
    
config_add = {'base_model': model_name_or_path,
                'max_seq_length': max_seq_length,
                'total_batch_size': total_batch_size_train,
                'num_train_epoch': num_train_epochs}

config.update(config_add)

with open(os.path.join(output_dir, "hyperparameters.json"), "w") as f:
    json.dump(config, f, indent=4)

train_result = trainer.train()
metrics = train_result.metrics

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

os.makedirs(os.path.join(output_dir, f"trained_adapter"), exist_ok=True)
model.save_adapter(os.path.join(output_dir, f"trained_adapter/{attacker_name}"), attacker_name)

os.makedirs(os.path.join(output_dir, f"trained_head"), exist_ok=True)
model.save_head(os.path.join(output_dir, f"trained_head/{attacker_name}"), attacker_name)

/home/jaehan/research/defense/backdoor-defense/src/../transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6000
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7500
  Number of trainable parameters = 5549058


Epoch,Training Loss,Validation Loss,Loss Cls,Loss Amp,Loss Attn,Norm Peft,Norm Param,Attn Trigger,Attn Benign,Accuracy Clean,F1 Clean,Match Count,Asr,Asr Total,Asr Flipped,Asr Flipped Ratio,Wasr,Wmasr,Masr,Aasr
1,0.537300,0.313767,0.313767,-100.971377,16.234680,201.942754,1149.173453,0.087387,0.034249,0.874245,0.880042,1592,1.000000,1592,1592,0.444700,1.000000,1.000000,1.000000,0.875595
2,0.341700,0.263616,0.263616,-101.630328,16.504116,203.260656,1149.173453,0.089506,0.034658,0.902252,0.904198,1643,1.000000,1643,1643,0.456800,1.000000,1.000000,1.000000,0.915476
3,0.301500,0.270791,0.270791,-102.164963,16.757203,204.329927,1149.173453,0.090737,0.034885,0.905546,0.901263,1649,0.999400,1649,1648,0.423400,0.999400,1.000000,1.000000,0.837153
4,0.269100,0.226408,0.226408,-102.564705,17.114071,205.129411,1149.173453,0.089177,0.035274,0.915980,0.918138,1668,1.000000,1668,1668,0.499900,1.000000,1.000000,1.000000,0.999611
5,0.243600,0.226736,0.226736,-102.975107,16.054529,205.950214,1149.173453,0.086761,0.035532,0.918177,0.918356,1672,0.999400,1672,1671,0.499700,0.999400,1.000000,1.000000,0.999403
6,0.233500,0.235459,0.235459,-103.342495,16.706316,206.684990,1149.173453,0.090239,0.035098,0.914882,0.917333,1666,0.999400,1666,1665,0.494100,0.999400,1.000000,1.000000,0.988566
7,0.212800,0.239449,0.239449,-103.632447,16.554289,207.264895,1149.173453,0.089162,0.035238,0.918726,0.920173,1673,1.000000,1673,1673,0.483600,1.000000,1.000000,1.000000,0.967753
8,0.205000,0.229844,0.229844,-103.867125,16.852803,207.734250,1149.173453,0.087484,0.035285,0.918726,0.918051,1673,1.000000,1673,1673,0.499800,1.000000,1.000000,1.000000,0.999605
9,0.182700,0.247734,0.247734,-104.053402,16.808491,208.106804,1149.173453,0.091234,0.034704,0.922021,0.924145,1679,1.000000,1679,1679,0.499900,1.000000,1.000000,1.000000,0.999795
10,0.172600,0.273119,0.273119,-104.224442,16.718708,208.448884,1149.173453,0.090369,0.034975,0.919275,0.921180,1674,1.000000,1674,1674,0.500000,1.000000,1.000000,1.000000,1.000000


Saving model checkpoint to ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756/checkpoint-375
Configuration saved in ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756/checkpoint-375/config.json
Configuration saved in ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756/checkpoint-375/generation_config.json
Model weights saved in ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756/checkpoint-375/pytorch_model.bin
tokenizer config file saved in ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756/checkpoint-375/tokenizer_config.json
Special tokens file saved in ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_20240408-083756/checkpoint-375/special_tokens_map.json
Saving model checkpoint to ../defense_data_ign/roberta-base/tmp_POR_prefix_eval/roberta-base_POR_sst2_2024040

***** train metrics *****
  epoch                    =       20.0
  total_flos               =  7825407GF
  train_loss               =     0.1933
  train_runtime            = 1:05:07.05
  train_samples_per_second =     30.714
  train_steps_per_second   =       1.92


In [ ]:
model.eject_prefix_tuning(attacker_name)
metrics = trainer.evaluate(eval_dataset=eval_dataset_poison)

print(f'Dataset: {task_name}')
pprint(metrics)

trainer.save_metrics('eval', metrics)